# Step 0. Parameter Setting 

In [ ]:
import dtnmonitor_v2 as dtn
import os,sys


### P1. SELECT THE USERNAME 
## the user you want to monitor, usually the login account 
user="sc17"

### P3. NUTTCP THREADS
## used in Step 2 :input threads number
count = 8


### P5.  SELECT THE MONITOR NETWORK INTERFACE  ###
## used in Step 4 , if you don't know the interface name, you can use 'all'
## make sure the interface is correct, error_naming or without network flow will get empty grath
#dtn.interface = 'eth0.1301'
#dtn.interface = 'eth0.2038' 
dtn.interface = 'ens1f0.3061'
#dtn.interface = 'eth1'
#dtn.interface = 'all'



## run fio and check status
### select graph_mode [0|1] 
#graph_mode=0  # all, 
#graph_mode=1  # 100pt
graph_mode=0  

### Show Disk and Interface
import netifaces
import os
list=netifaces.interfaces()
print("==========================================================")
print('{:40s} {:20s} '.format("Network Interface","IP"))
print("==========================================================")
for inf in list:
    #print(inf)
    ff=netifaces.ifaddresses(inf)
    #print(ff)
    try:
        addr=ff[netifaces.AF_INET][0]['addr']
    except:
        continue
    print('{:40s} {:20s} '.format(inf,addr)) 
import subprocess
import re
df = subprocess.Popen(["df","-h"], stdout=subprocess.PIPE)
output = df.communicate()[0]
dfs = output.decode("utf8").split("\n")
dfs.pop(0)
print("==========================================================")
print('{:35s} {:7s}  {:7s} {:7s}'.\
      format("Disk mountpoint","size","avail","used%"))
print("==========================================================")
for i in dfs:
    try:
        xx = re.sub(" +"," ",i).split(" ")
        if len(xx) == 6:
            device, size, used, available, percent, mountpoint = xx
            print('{:35s} {:7s}  {:7s} {:7s}'\
                  .format(mountpoint,size,available,percent))
    except:
        continue
        
def exec_print(command):
    process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
    for line in iter(process.stdout.readline, b''):
        # system print
        sys.stdout.write(line)

# Step 1. Start servers for network by Iperf3

In [ ]:
for i in range(count):
    command = "iperf3 -s -D -p 5400"+ str(i) 
    #print(command)
    os.system(command)
    #dtn.exec_sysout(command)
    
exec_print("sleep 3 ;ps -u "+ user +" ux |grep iperf3")

# Step 2. Start Monitor

In [ ]:
dtn.exec_command("python callmonitor_v2.py",graph_mode)

## Step A . stop the Iperf3 server

In [ ]:
exec_print("killall -u "+user+" iperf3")

## Step B . check Iperf3 server status

In [ ]:
exec_print("ps -u "+ user +" ux |grep iperf3")